<a href="https://colab.research.google.com/github/ralbu85/DataScience_2022S/blob/master/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EC%88%98%EC%97%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 불러오기

In [2]:
import pandas as pd

df = pd.read_csv('auto.csv')
df

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,18.0
1,8,350.0,165.0,3693.0,11.5,70,1,15.0
2,8,318.0,150.0,3436.0,11.0,70,1,18.0
3,8,304.0,150.0,3433.0,12.0,70,1,16.0
4,8,302.0,140.0,3449.0,10.5,70,1,17.0
...,...,...,...,...,...,...,...,...
387,4,140.0,86.0,2790.0,15.6,82,1,27.0
388,4,97.0,52.0,2130.0,24.6,82,2,44.0
389,4,135.0,84.0,2295.0,11.6,82,1,32.0
390,4,120.0,79.0,2625.0,18.6,82,1,28.0


In [3]:
df2 = pd.get_dummies(df, columns=['origin'], dtype=int) #원핫인코딩
df2

,cylinders,displacement,horsepower,weight,acceleration,model_year,mpg,origin_1,origin_2,origin_3
0,8,307.0,130.0,3504.0,12.0,70,18.0,1,0,0
1,8,350.0,165.0,3693.0,11.5,70,15.0,1,0,0
2,8,318.0,150.0,3436.0,11.0,70,18.0,1,0,0
3,8,304.0,150.0,3433.0,12.0,70,16.0,1,0,0
4,8,302.0,140.0,3449.0,10.5,70,17.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
387,4,140.0,86.0,2790.0,15.6,82,27.0,1,0,0
388,4,97.0,52.0,2130.0,24.6,82,44.0,0,1,0
389,4,135.0,84.0,2295.0,11.6,82,32.0,1,0,0
390,4,120.0,79.0,2625.0,18.6,82,28.0,1,0,0


In [4]:
## X와 y나누기
y = df2['mpg'] # 종속변수
X = df2.drop(columns=['mpg']) # 독립변수

## 교차검증(수동)

In [12]:
mse_result, r2_result = [], []

from sklearn.linear_model import LinearRegression # 학습할 모델
from sklearn.metrics import mean_squared_error, r2_score # 모델의 평가
from sklearn.model_selection import KFold # 교차검증위해 데이터 쪼개기

kfold = KFold(n_splits = 3, shuffle=True) # 집단을 3개로 나눠서 3번 반복하는 객체 생성

for train_idx, test_idx in kfold.split(X,y):
    print('='*50)
    ## 데이터셋 나누기
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    ## 훈련데이터로 모델 학습
    model = LinearRegression() # 회귀모델 객체 생성
    model.fit(X_train, y_train) # 훈련데이터로 학습
    ## 테스팅데이터로 검증
    y_pred = model.predict(X_test) # 예측
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    ## 테스팅데이터에 대한 성능을 삽입
    mse_result.append(mse)
    r2_result.append(r2)

mse_result, r2_result

([12.828457080551084, 11.65555629558054, 10.215017094277664],
 [0.7819660822899352, 0.7941930106884127, 0.8464472979001824])

## sklearn의 cross_validate 사용

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_validate

model = LinearRegression() # 학습킬 객체
# model = KNeighborsRegressor(n_neighbors=10)
kfold = KFold(n_splits=5, shuffle=True) # 데이터를 분할할 객체
metrics = ['neg_mean_squared_error','r2'] # 성능검증 지표

result = cross_validate(estimator=model, # 학습시킬 객체
                        X=X, y=y, # 데이터
                        cv=kfold, # 데이터 분할 객체
                        scoring=metrics,
                        return_train_score = True
                        ) # 성능지표

result = pd.DataFrame(result)

In [35]:
result.mean()

fit_time                         0.003863
score_time                       0.002144
test_neg_mean_squared_error    -11.226677
train_neg_mean_squared_error   -10.621908
test_r2                          0.815407
train_r2                         0.825188
dtype: float64